In [1]:
import augur
from augur.PiperReader import PiperMail

# import everything from githubapi.py and ghtorrent.py so we can
# just copy and paste our function later
import json
import pandas as pd
from perceval.backends.core.pipermail import Pipermail
import perceval
import os, os.path

# create an Augur application so we can test our function
print(os.getcwd())
print(perceval.__path__)
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
print(os.getcwd())
link = "https://lists.opendaylight.org/pipermail/"
#mail = ["aalldp-dev","alto-dev","archetypes-dev"]
mail = ["aalldp-dev","archetypes-dev","alto-dev"]
#print(os.getcwd())
file = "opendaylight-"
if "notebooks" in os.getcwd():
    os.chdir("..")
path = "/augur/data/opendaylight-" 
for x in range(len(mail)):
    #print(link+mail[x])
    #print(os.getcwd())
    #print(os.path.join(os.getcwd() + path+'.json'))
    place = os.path.join(os.getcwd() + path + mail[x] +'.json')
    if(os.path.exists(place)):
        print(mail[x])
        print("File Exists")
        continue                   
    repo = Pipermail(url = "https://lists.opendaylight.org/pipermail/"+ mail[x] + "/",dirpath="tmp/archives"+mail[x])
    #print(repo)
    outfile = open(place,"w+")
    for message in repo.fetch():
        obj = json.dumps(message, indent=4, sort_keys=True)
        outfile.write(obj)
        outfile.write('\n')
    outfile.close()
    print("Created File",mail[x])
print("Finished downloading files")
print(link)

2018-06-18 10:02:48 keanu-Inspiron-5567 perceval.backends.core.pipermail[7516] INFO Looking for messages from 'https://lists.opendaylight.org/pipermail/alto-dev/' since 1970-01-01 00:00:00+00:00
2018-06-18 10:02:48 keanu-Inspiron-5567 perceval.backends.core.pipermail[7516] INFO Downloading mboxes from 'https://lists.opendaylight.org/pipermail/alto-dev/' to since 1970-01-01 00:00:00+00:00


/home/keanu/temp/augur_push/augur1/augur/notebooks
_NamespacePath(['/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/perceval'])
/home/keanu/temp/augur_push/augur1/augur/notebooks
aalldp-dev
File Exists
archetypes-dev
File Exists


2018-06-18 10:03:26 keanu-Inspiron-5567 perceval.backends.core.pipermail[7516] INFO 35/35 MBoxes downloaded
2018-06-18 10:03:26 keanu-Inspiron-5567 perceval.backends.core.mbox[7516] INFO Done. 536/536 messages fetched; 0 ignored
2018-06-18 10:03:26 keanu-Inspiron-5567 perceval.backends.core.pipermail[7516] INFO Fetch process completed


Created File alto-dev
Finished downloading files
https://lists.opendaylight.org/pipermail/


In [2]:
print(link)
temp = augurApp.piper()
temp.make(link)
print(temp)

https://lists.opendaylight.org/pipermail/
ugh
wtf
https://lists.opendaylight.org/pipermail/
Hey
File uploaded  2
File uploaded  2
File uploaded  855
True
Finished
